## Data Asistencia a conciertos: archivo .csv

Como tercera fuente he decididio entrar en la web del 'Ministerio del Gobierno de España', concretamemte en el apartado de 'Cultura y deporte'. De esta he descargado datos en un csv, los cuales he arreglado antes de subir al jupyter, ya que no tenían los datos como a mi me gustaría. 
A partir de ahí, me he descargado estos datos en un archivo .csv sobre la el porcentaje de asistencia a conciertos (en % del total de cada colectivo) personas según la asistencia a conciertos por Comunidad Autónoma y tamaño del municipio, en concreto, en la Comunidad de Madrid.

Los datos se recogen de una 'Encuesta de Hábitos y Prácticas Culturales' acotados en 2021-2022, cuya url es 'https://estadisticas.mecd.gob.es/CulturaJaxiPx/Tabla.htm?path=/t9/p9/a1a2021-2022/c05//l0/&file=05081.px&type=pcaxis&L=0'


En cuanto al proceso de limpieza que se lleva a cabo, se ordenan los datos y se ajustan a un formato más equitativo para poder ser compatible con el resto de los datos del trabajo. 

De esta forma, cuanto más limpio queda cada uno de los dataframes, más sencillo será posteriormente ver la relación entre unos y otros, en la nueva base de datos posteriormente creada. 

In [1]:
import pandas as pd
import numpy as np
import statistics as stats

import warnings
warnings.filterwarnings('ignore')

import pylab as plt
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
asistencia = pd.read_csv('../Data/CollectedOnline/asistencia-conciertos.csv', sep=';')

In [3]:
asistencia

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Comunidad de Madrid,NaN,NaN,NaN
1,Asistencia (2021-2022),conciertos,musica_clasica,Musica_actual
2,Total han asistido en el último año,"16,7","5,6","12,6"
3,En los últimos tres meses,"9,5","2,3","7,6"
4,Entre tres meses y un año,"7,2","3,3",5
5,Entre tres meses y un año,"7,2","3,3",5
6,Nunca o casi nunca,33,"61,4","38,3"


1. PRIMER PASO: 
- Cambio el nombre de las columnas, en este caso serán los datos de la fila con index=2.
- Se eliminan las filas que no son útiles.
- Reseteo el index (y elimino la columna que se ha creado cuando reseteo el index).

In [4]:
nombres_viejos = asistencia.columns

In [5]:
nombres_viejos

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'], dtype='object')

In [6]:
nombres_nuevos = list(asistencia.iloc[1,:])

nombres_nuevos

['Asistencia (2021-2022)', 'conciertos', 'musica_clasica', 'Musica_actual']

In [7]:
rename_dict = dict(zip(nombres_viejos,nombres_nuevos))
rename_dict

{'Unnamed: 0': 'Asistencia (2021-2022)',
 'Unnamed: 1': 'conciertos',
 'Unnamed: 2': 'musica_clasica',
 'Unnamed: 3': 'Musica_actual'}

In [8]:
asistencia.rename(columns=rename_dict,inplace=True)

In [9]:
asistencia

,Asistencia (2021-2022),conciertos,musica_clasica,Musica_actual
0,Comunidad de Madrid,NaN,NaN,NaN
1,Asistencia (2021-2022),conciertos,musica_clasica,Musica_actual
2,Total han asistido en el último año,"16,7","5,6","12,6"
3,En los últimos tres meses,"9,5","2,3","7,6"
4,Entre tres meses y un año,"7,2","3,3",5
5,Entre tres meses y un año,"7,2","3,3",5
6,Nunca o casi nunca,33,"61,4","38,3"


In [10]:
data = asistencia.iloc[2:,:]

data

,Asistencia (2021-2022),conciertos,musica_clasica,Musica_actual
2,Total han asistido en el último año,"16,7","5,6","12,6"
3,En los últimos tres meses,"9,5","2,3","7,6"
4,Entre tres meses y un año,"7,2","3,3",5
5,Entre tres meses y un año,"7,2","3,3",5
6,Nunca o casi nunca,33,"61,4","38,3"


In [11]:
data.reset_index(inplace=True)

In [12]:
data.head()

,index,Asistencia (2021-2022),conciertos,musica_clasica,Musica_actual
0,2,Total han asistido en el último año,"16,7","5,6","12,6"
1,3,En los últimos tres meses,"9,5","2,3","7,6"
2,4,Entre tres meses y un año,"7,2","3,3",5
3,5,Entre tres meses y un año,"7,2","3,3",5
4,6,Nunca o casi nunca,33,"61,4","38,3"


In [13]:
data.drop(columns="index",inplace=True)

In [15]:
data

,Asistencia (2021-2022),conciertos,musica_clasica,Musica_actual
0,Total han asistido en el último año,"16,7","5,6","12,6"
1,En los últimos tres meses,"9,5","2,3","7,6"
2,Entre tres meses y un año,"7,2","3,3",5
3,Entre tres meses y un año,"7,2","3,3",5
4,Nunca o casi nunca,33,"61,4","38,3"


2. SEGUNDO PASO:
- Inserto una nueva columna, que aunque será igual en todas las filas, puede ser de utilidad para unir a otras tablas. 

In [16]:
data.insert(0, 'Nueva_Columna', 'Madrid')

In [17]:
data

,Nueva_Columna,Asistencia (2021-2022),conciertos,musica_clasica,Musica_actual
0,Madrid,Total han asistido en el último año,"16,7","5,6","12,6"
1,Madrid,En los últimos tres meses,"9,5","2,3","7,6"
2,Madrid,Entre tres meses y un año,"7,2","3,3",5
3,Madrid,Entre tres meses y un año,"7,2","3,3",5
4,Madrid,Nunca o casi nunca,33,"61,4","38,3"


In [18]:
nombres_nuevos = ['comunidad', 'asistencia(2021-2022)', 'conciertos', 'clasica', 'actual']

data.columns = nombres_nuevos

In [19]:
data

,comunidad,asistencia(2021-2022),conciertos,clasica,actual
0,Madrid,Total han asistido en el último año,"16,7","5,6","12,6"
1,Madrid,En los últimos tres meses,"9,5","2,3","7,6"
2,Madrid,Entre tres meses y un año,"7,2","3,3",5
3,Madrid,Entre tres meses y un año,"7,2","3,3",5
4,Madrid,Nunca o casi nunca,33,"61,4","38,3"


In [20]:
asistencia = data

In [21]:
asistencia

,comunidad,asistencia(2021-2022),conciertos,clasica,actual
0,Madrid,Total han asistido en el último año,"16,7","5,6","12,6"
1,Madrid,En los últimos tres meses,"9,5","2,3","7,6"
2,Madrid,Entre tres meses y un año,"7,2","3,3",5
3,Madrid,Entre tres meses y un año,"7,2","3,3",5
4,Madrid,Nunca o casi nunca,33,"61,4","38,3"


3. TERCER PASO:
- Importo mi tabla a .csv

Una vez he importado los dataframe como .csv los marco para que no se me vuelva a convertir.

In [30]:
#asistencia.to_csv('../Data/Datos_Clean/asistencia.csv', index=False)